In [1]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

In [2]:
# %reset -f

In [3]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Input/current_g3x_con.xlsx'
x1 = pd.ExcelFile(filename) # check this out!

In [4]:
# res = len(x1.sheet_names)
sheetList = x1.sheet_names
sheetList.sort()
sheetList

['1st Qtr 2019 Data',
 '2018 Monthly Data',
 '2019 Monthly Data',
 '2nd Qtr 2019 Data',
 'BC QUICK LOOK',
 'Cal G3X Master',
 'G3X Master']

In [5]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_AVI_TSN_Incentive.txt'
df_tierMap_TSN = pd.read_csv(filename2, sep = '\t')
df_tierMap_TSN.columns = df_tierMap_TSN.columns.str.replace(' ', '_')

filename3 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_AVI_NonTSN_Incentive.txt'
df_tierMap_nonTSN = pd.read_csv(filename3, sep = '\t')
df_tierMap_nonTSN.columns = df_tierMap_nonTSN.columns.str.replace(' ', '_')

# df.shape

In [6]:
for sheet in sheetList:
    if '1st' in sheet and 'Qtr' in sheet:
        print('Q1 exist')
        df1 = pd.read_excel(filename,sheet_name=sheet,header=1)
        
        df1.columns = df1.columns.str.replace(' ', '_')
        df1.columns = df1.columns.str.replace('#', 'num')
        
        df_current = df1
        
    elif '2nd' in sheet and 'Qtr' in sheet:
        print('Q2 exist')
        df2 = pd.read_excel(filename,sheet_name=sheet,header=1)
        df2.columns = df2.columns.str.replace(' ', '_')
        df2.columns = df2.columns.str.replace('#', 'num')
        
        df_historic1 = df1.iloc[:,[1,14,15,16,17,18,19,20]]
        df_historic1 = df_historic1.add_suffix('_Q1')
        
        df_current = df2
        
    elif '3rd' in sheet and 'Qtr' in sheet:
        print('Q3 exist')
        df3 = pd.read_excel(filename,sheet_name=sheet,header=1)
        df3.columns = df3.columns.str.replace(' ', '_')
        df3.columns = df3.columns.str.replace('#', 'num')
        
        df_historic2 = df2.iloc[:,[1,14,15,16,17,18,19,20]]
        df_historic2 = df_historic2.add_suffix('_Q2')
        
        df_current = df3
        
    elif '4th' in sheet and 'Qtr' in sheet:
        print('Q4 exist')
        df4 = pd.read_excel(filename,sheet_name=sheet,header=1)
        df4.columns = df4.columns.str.replace(' ', '_')
        df4.columns = df4.columns.str.replace('#', 'num')
            
        df_historic3 = df3.iloc[:,[1,14,15,16,17,18,19,20]]
        df_historic3 = df_historic3.add_suffix('_Q3')
        
        df_current = df4

Q1 exist
Q2 exist


In [7]:
# df_current.columns.to_series(index=range(df_current.shape[1]))

In [8]:
currentYear = datetime.now().year

In [9]:
tup_now = datetime.now().timetuple()

In [10]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
# print(dt_currentData)
ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

In [11]:
calculatedData = pd.DataFrame([])
for ii in range(df_current.shape[0]):

    singleCusty = df_current.iloc[[ii]]
    singleCusty.reset_index(inplace=True)

    singleCusty_select = singleCusty[
        ['G3X_CO_Name','TSN','Net_Shipments_YTD','num_Locations']]

    tt = pd.to_datetime(singleCusty['Last_Transaction'])[0].timetuple()
#     excel_date = singleCusty.Last_Transaction
    
#     date_lt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + excel_date - 2)
#     tt = date_lt.timetuple()
        
    if  tt.tm_year == currentYear:

        if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
            daysINyear = 366
            print('Leap year')
        else:
            daysINyear = 365     

        numDays = tt.tm_yday # get number of days

        estimatedTires = np.floor(singleCusty_select.Net_Shipments_YTD.astype(int)/(numDays/daysINyear)).values
        numLocations = singleCusty.num_Locations.values
#         netShip = singleCusty_select.Net_Shipments.values.astype(int)
#         netGYc = singleCusty_select.avi_Core_Net_Shipments.values.astype(int)
#         netGYm = singleCusty_select.avi_Mid_Net_Shipments.values.astype(int)
#         netKelly = singleCusty_select.Kelly_Net_Shipments.values.astype(int)

        # Check if customer is aligned
        if singleCusty_select.TSN.values == 'N':
            df_tierMap = df_tierMap_nonTSN
            if   (estimatedTires >= (df_tierMap.iloc[0,2] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[0,3] * numLocations)):
                estTier = 0
            else:
                estTier = 1
            if estTier < 1:
                tierUP = estTier + 1
            else:
                tierUP = 1
        else:
            df_tierMap = df_tierMap_TSN
            if   (estimatedTires >= (df_tierMap.iloc[0,2] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[0,3] * numLocations)):
                estTier = 0
            elif (estimatedTires >= (df_tierMap.iloc[1,2] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[1,3] * numLocations)):
                estTier = 1
            else:
                estTier = 2
            if estTier < 2:
                tierUP = estTier + 1
            else:
                tierUP = 2
    

            
        # floor to underestimate estimated tires
        avi_est_tires_rebate = estimatedTires * df_tierMap.iloc[estTier,1]
        avi_1up_tires = df_tierMap.iloc[tierUP,2]
        avi_1up_tires_rebate = avi_1up_tires * df_tierMap.iloc[tierUP,1]

        if avi_est_tires_rebate >= avi_1up_tires_rebate*numLocations:
            avi_1up_tires = estimatedTires
            avi_1up_tires_rebate = avi_est_tires_rebate

        range_dollars_ll = ((avi_1up_tires*df_tierMap.iloc[0,1]))#.values
        range_dollars_ul = ((avi_1up_tires*df_tierMap.iloc[tierUP,1]))#.values
   

     
###########################################################################################################################        
#         print(singleCusty_select)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [estTier])        
#         print('\nEstimated Tires ~ avi:', estimatedTires)        
#         print('\nEstimated Rebate ~ avi:',avi_est_tires_rebate)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [tierUP])        
#         print('\n1up Tires ~ avi:', avi_1up_tires*numLocations)        
#         print('\n1up Dollars ~ avi:',avi_1up_tires_rebate*numLocations)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul*numLocations, '\n')
# ###########################################################################################################################
#         print(tt, '\n' 'indexNumber:', ii, '\n')
###########################################################################################################################

        tempData = pd.DataFrame(
            {'estimated_tier':estTier,
             'current_est_tires':estimatedTires, 'total_est_rebate':avi_est_tires_rebate,
             'Up_tier':tierUP,
             'avi_up_tires':avi_1up_tires*numLocations, 
             'avi_up_rebate':avi_1up_tires_rebate*numLocations,'grain': 'Annual'})
        calculatedData = calculatedData.append(tempData)
    else:
#         print('\nJUNK:', tt,'\n', 'indexNumber:', ii)
        tempData = pd.DataFrame(
            {'estimated_tier':[0],
             'current_est_tires':[0], 'total_est_rebate':[0],
             'Up_tier':[1],
             'avi_up_tires':[0], 'avi_up_rebate':[0], 'grain': ['Annual']})
        calculatedData = calculatedData.append(tempData)
#         print(ii)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df_current], axis=1)

In [12]:
tt.tm_year

2019

In [13]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
finalData.head()

,estimated_tier,current_est_tires,total_est_rebate,Up_tier,avi_up_tires,avi_up_rebate,grain,Dealer_List,G3X_Common_Owner,Unnamed:_2,...,Goodyear_Core_Net_Purchases_QTD,Kelly_Net_Purchases_QTD,Other_Net_Purchases_QTD,Incentive_Payment,AVI_Curr_Tier,Net_Shipments_YTD,Units_to_AVI_Nxt_Tier,Goodyear_Core_Net_Purchases_YTD,AVI_Payment,Last_Transaction
0,0,414.0,0.0,1,1000.0,2000.0,Annual,YOUNGBLOOD AUTOMOTIVE & TIRE_00A0000722,00A0000722,1,...,22,0,2,0,0,201,799,21,0,2019-06-26
1,0,857.0,0.0,1,1000.0,2000.0,Annual,WITT BOYS INC_00A0005385,00A0005385,1,...,107,85,37,1499,0,425,575,166,0,2019-06-30
2,0,1851.0,0.0,1,2000.0,4000.0,Annual,WILLEYS TIRE SHOP_00A0006972,00A0006972,1,...,252,145,62,3179,0,918,1082,424,0,2019-06-30
3,0,1389.0,0.0,1,2000.0,4000.0,Annual,WRIGHT BROTHERS AUTO SERVICE_00A0008242,00A0008242,1,...,224,59,47,1945,0,674,1326,407,0,2019-06-26
4,0,454.0,0.0,1,1000.0,2000.0,Annual,WITTHOFT CORP INC_00B0146291,00B0146291,1,...,23,10,30,291,0,223,777,42,0,2019-06-28


In [15]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_G3X_AVI.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_G3X_AVI_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
# path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
# writeFilename = path + 'cmp_G3X.csv'
# finalData.to_csv(writeFilename, index=False)
# writeFilename_archive = path + 'csv_archive/' + str(dt_currentData.year) + '_' + str(dt_currentData.month) + '_' + str(dt_currentData.day) + '_' + 'CMP_G3X.csv'
# finalData.to_csv(writeFilename_archive, index=False)